In [1]:
from transformers import RobertaTokenizer, RobertaModel
import torch

# Initialize tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('macedonizer/mk-roberta-base')
model = RobertaModel.from_pretrained('macedonizer/mk-roberta-base')

Some weights of the model checkpoint at macedonizer/mk-roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at macedonizer/mk-roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to b

In [2]:
import pandas as pd 

data = pd.read_csv('../../preprocessed_data.csv')

In [3]:
# Create sentences

# Assuming 'df' is your DataFrame
data['word'] = data['word'].astype(str)
grouped_sentences = data.groupby('sentence_id')['word'].apply(lambda words: ' '.join(words)).reset_index(name='sentence')

print(grouped_sentences.head())

   sentence_id                                           sentence
0            1  Беше јасен и студен априлски ден а часовниците...
1            2  Винстон Смит со брадата спуштена на градите за...
2            3  Ходникот баздеше на варена зелка и на стари пр...
3            4  На неговиот крај висеше постер во боја премног...
4            5  Тој прикажуваше само едно огромно лице широко ...


In [47]:
import numpy as np

embeddings_data = []

data['word'] = data['word'].astype(str)

grouped = data.groupby('sentence_id')['word'].apply(list).reset_index(name='sentence_words')
print(grouped)


      sentence_id                                     sentence_words
0               1  [Беше, јасен, и, студен, априлски, ден, а, час...
1               2  [Винстон, Смит, со, брадата, спуштена, на, гра...
2               3  [Ходникот, баздеше, на, варена, зелка, и, на, ...
3               4  [На, неговиот, крај, висеше, постер, во, боја,...
4               5  [Тој, прикажуваше, само, едно, огромно, лице, ...
...           ...                                                ...
6785         6786  [Интересите, на, престижот, го, правеа, пожелн...
6786         6787  [Разни, писатели, како, Шекспир, Милтон, Свифт...
6787         6788  [Овие, преводи, беа, бавна, и, тешка, работа, ...
6788         6789  [Постоеја, исто, така, големи, количества, од,...
6789         6790  [Токму, за, да, се, обезбеди, време, за, прели...

[6790 rows x 2 columns]


In [51]:
# i = 0

# for _, row in grouped.head(20).iterrows():
#     if i == 20:
#         break
#     sentence_id = row['sentence_id']
#     words = row['sentence_words']
#     sentence = " ".join(words)
    
#     inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512)
#     with torch.no_grad():
#         outputs = model(**inputs)
    
#     sentence_embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

#     word_embeddings = []
#     current_word_index = 0
#     subword_embeddings = []

#     tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'].squeeze().tolist())

#     for token_index, token in enumerate(tokens):
#         if token in [tokenizer.cls_token, tokenizer.sep_token, tokenizer.pad_token]:
#             continue  # Skip special tokens
        
#         # Check if the current token is a part of the current word
#         if current_word_index < len(words) and words[current_word_index] in sentence[sentence.find(token):]:
#             subword_embeddings.append(outputs.last_hidden_state[0, token_index, :].numpy())
#             sentence = sentence.replace(token, "", 1)
#         else:
#             # Average the embeddings for the previous word
#             if subword_embeddings:
#                 word_embedding = np.mean(subword_embeddings, axis=0)
#                 word_embeddings.append(word_embedding)
#                 subword_embeddings = []  # Reset for the next word
            
#             current_word_index += 1  # Move to the next word
            
#             # Start collecting embeddings for the next word
#             if current_word_index < len(words) and words[current_word_index] in sentence[sentence.find(token):]:
#                 subword_embeddings.append(outputs.last_hidden_state[0, token_index, :].numpy())
#                 sentence = sentence.replace(token, "", 1)
    
#     if subword_embeddings:
#         word_embedding = np.mean(subword_embeddings, axis=0)
#         word_embeddings.append(word_embedding)

#     # Store the results
#     word_ids = data[data['sentence_id'] == sentence_id]['word_id'].tolist()
#     for word_id, word_embedding in zip(word_ids, word_embeddings):
#         embeddings_data.append({
#             'sentence_id': sentence_id,
#             'word_id': word_id,
#             'sentence_embedding': sentence_embedding.tolist(),
#             'word_embedding': word_embedding.tolist()
#         })
        
#     print(f"Processing sentence_id {sentence_id} with {len(words)} words, generated {len(word_embeddings)} word embeddings.")

#     i += 1

# embeddings_df = pd.DataFrame(embeddings_data)


In [52]:
embeddings_data = []

for _, row in grouped.iterrows():
    sentence_id = row['sentence_id']
    words = row['sentence_words']
    sentence = " ".join(words)

    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    
    sentence_embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
    
    # Get all token embeddings
    all_token_embeddings = outputs.last_hidden_state.squeeze().numpy()

    # Initialize variables to track word formation from tokens
    word_embeddings = []
    token_idx = 1  # Start after [CLS]
    for word in words:
        # Accumulate token embeddings for the current word
        token_embeddings = []
        while token_idx < len(all_token_embeddings) - 1:  # Exclude [SEP]
            token_embeddings.append(all_token_embeddings[token_idx])
            token_text = tokenizer.decode(inputs['input_ids'][0, token_idx])
            
            # Condition to break the loop if this token marks the completion of the word
            if token_text == word or tokenizer.decode(inputs['input_ids'][0, token_idx + 1]).startswith(' '):
                token_idx += 1
                break
            token_idx += 1
        
        # Average the embeddings for the current word
        if token_embeddings:
            word_embeddings.append(np.mean(token_embeddings, axis=0))

    word_ids = data[data['sentence_id'] == sentence_id]['word_id'].tolist()
    for word_id, word_embedding in zip(word_ids, word_embeddings):
        embeddings_data.append({
            'sentence_id': sentence_id,
            'word_id': word_id,
            'sentence_embedding': sentence_embedding.tolist(),
            'word_embedding': word_embedding.tolist()
        })


In [53]:
embeddings_df = pd.DataFrame(embeddings_data)

In [54]:
embeddings_df

,sentence_id,word_id,sentence_embedding,word_embedding
0,1,1,"[-0.6169265508651733, 0.34044384956359863, 0.9...","[-0.19491949677467346, 0.3511471450328827, 1.0..."
1,1,2,"[-0.6169265508651733, 0.34044384956359863, 0.9...","[-0.9175711870193481, 0.2979837656021118, -0.7..."
2,1,3,"[-0.6169265508651733, 0.34044384956359863, 0.9...","[0.10833288729190826, -0.8593301177024841, 1.0..."
3,1,4,"[-0.6169265508651733, 0.34044384956359863, 0.9...","[-1.1452068090438843, 0.9079347252845764, 0.48..."
4,1,5,"[-0.6169265508651733, 0.34044384956359863, 0.9...","[-0.4176254868507385, -0.5781083106994629, 2.3..."
...,...,...,...,...
96057,6790,393,"[0.3545113801956177, 0.2402992844581604, 0.740...","[0.12584611773490906, 1.1002881526947021, 0.75..."
96058,6790,146,"[0.3545113801956177, 0.2402992844581604, 0.740...","[2.428894519805908, -0.22653913497924805, -0.0..."
96059,6790,418,"[0.3545113801956177, 0.2402992844581604, 0.740...","[2.666874408721924, -0.07137368619441986, 0.47..."
96060,6790,4723,"[0.3545113801956177, 0.2402992844581604, 0.740...","[1.2628905773162842, 0.04624204337596893, 0.66..."


In [6]:
# embeddings_df.to_csv('../../embeddings_data.csv', index=False)

In [55]:
df_word_embeddings = embeddings_df[['word_id', 'sentence_id', 'word_embedding']].drop_duplicates(subset=['word_id'])

df_word_embeddings.reset_index(drop=True, inplace=True)

In [56]:
df_word_embeddings

,word_id,sentence_id,word_embedding
0,1,1,"[-0.19491949677467346, 0.3511471450328827, 1.0..."
1,2,1,"[-0.9175711870193481, 0.2979837656021118, -0.7..."
2,3,1,"[0.10833288729190826, -0.8593301177024841, 1.0..."
3,4,1,"[-1.1452068090438843, 0.9079347252845764, 0.48..."
4,5,1,"[-0.4176254868507385, -0.5781083106994629, 2.3..."
...,...,...,...
15940,15941,6789,"[0.25629445910453796, 0.03444233909249306, 0.9..."
15941,15942,6789,"[-0.8041114807128906, 0.5293155312538147, 1.09..."
15942,15943,6790,"[-0.5313321352005005, 0.3916394114494324, 1.94..."
15943,15944,6790,"[-0.8071935176849365, 0.21436543762683868, 0.8..."


In [57]:
df_sentence_embeddings = embeddings_df[['sentence_id', 'sentence_embedding']].drop_duplicates(subset=['sentence_id'])

df_sentence_embeddings.reset_index(drop=True, inplace=True)

In [58]:
df_sentence_embeddings

,sentence_id,sentence_embedding
0,1,"[-0.6169265508651733, 0.34044384956359863, 0.9..."
1,2,"[-0.31695646047592163, -0.06865434348583221, 1..."
2,3,"[-0.3484392464160919, -0.25705409049987793, 0...."
3,4,"[-0.048721782863140106, 0.7031247615814209, 1...."
4,5,"[0.6102833151817322, 0.14064306020736694, 1.02..."
...,...,...
6785,6786,"[0.2429082691669464, 0.22434552013874054, 1.11..."
6786,6787,"[0.47419729828834534, 0.10613614320755005, 1.1..."
6787,6788,"[0.004281379748135805, 0.016001710668206215, 1..."
6788,6789,"[0.47204872965812683, -0.029387781396508217, 0..."


In [59]:
df_word_embeddings.to_csv('../../word_embeddings.csv', index=False)
df_sentence_embeddings.to_csv('../../unique_sentence_embeddings.csv', index=False)